# FORM
[scipy]: https://scipy.org "Official website of the SciPy-project"

## TODO

```{todo}
Write this section.
```

## Syntax

```{eval-rst}
.. function:: flx.perform_FORM

    Syntax:
        ``flx.perform_FORM(LSF, SAMPLER, CONFIG)``

    Description:
        Perform a run of FORM (First Order Reliability Method).
        
    :param LSF: The limit-state function (that depends on the input random variables of sampler ``SAMPLER``).
    :type LSF: :type:`flxPara`
    :param SAMPLER: A sampler for a set (or multiple sets) of random variables.
    :type SAMPLER: :class:`flx.sampler`
    :param CONFIG: 
        A configuration dictionary. The following keys are allowed in ``CONFIG``:
    
         - ``u0`` or ``x0`` (*numpy.ndarray*): The start point of the optimization. For ``u0``, the start point is given in the underlying Standard Normal space. For ``x0``, the start point is given in the original space. By default, the zero-vector in standard Normal space is used as start point.
         - ``opt_method`` (*string*, default: ``"iHLRF"``): 
         
             A parameter that specifies the optimization algorithm to use. The following values are allowed:   
             
             - ``HLRF``: HLRF-algorithm
             - ``iHLRF``: iHLRF-algorithm
             
         - ``iHLRF_epsilon`` (*float*, default: 0.4): Parameter of the iHLRF-algorithm. Value must be positive.
         - ``iHLRF_lambda_start`` (*float*, default: 1.0): Parameter of the iHLRF-algorithm. Value must be positive.
         - ``iHLRF_reduce`` (*float*, default: 0.5): Parameter of the iHLRF-algorithm. Value must be positive.
         - ``max_iter`` (*int*, default: 100): The maximum number of iterations of the optimizer.
         - ``fd_method`` (*string*, default: ``"fwd_diff"``): 
         
             A parameter that specifies the finite-difference method to use. The following values are allowed:   
             
             - ``fwd_diff``: forward difference
             - ``cent_diff``: central difference
             - ``bwd_diff``: backward difference
             
         - ``fdstep`` (*float*, default: 1e-6): Factor for controlling the step size relative to the coordinate in original space. Value must be positive.
         - ``epsdyf`` (*float*, default: 2): Factor for controlling the step size relative to the coordinate transformed into standard Normal space. Value must be positive.
         - ``dxmin`` (*numpy.ndarray*, default: 1e-6 times vector of standard deviations of random variables): Maximum of finite difference step-size. The values of the elements of the vector must be positive.
         - ``eps1`` (*float*, default: 1e-5): Tolerance parameter for the first stopping criterion. Value must be positive.
         - ``eps2`` (*float*, default: 1e-5): Tolerance parameter for the second stopping criterion. Value must be positive.
         - ``only_partial`` (*bool*, default: *False*): Evaluate only the partial derivative at the start point.
         - ``dxdyAnalytical`` (*bool*, default: *True*): If ``True``, the derivative between original space and standard Normal space is evaluated analytically.
         - ``verboseLog`` (*bool*, default: *False*): Verbose logging.

    :type CONFIG: *dict*
    
    :returns:     
      A Python *dict* with the following information is returned:

        - ``FORM_beta`` (*float*): The reliability index estimated with FORM.
        - ``FORM_pf`` (*float*): The probability of failure associated with the linearized limit-state function.
        - ``pf_upper_bound`` (*float*): An upper bound for the probability of failure of the potentially non-linear limit-state function.
        - ``FROM_u`` (*numpy.ndarray*): Vector of the design-point in standard Normal space.
        - ``FROM_x`` (*numpy.ndarray*): Vector of the design-point in original Normal space.
        - ``N_lsf_calls`` (*int*): Total number of limit-state function calls required by FORM.N_lsf_calls
        - ``dzdy`` (*numpy.ndarray*): Partial derivative of the limit-state function with respect to standard Normal space (at the design point - or the start point(if ``only_partial==True``)).
        
    :rtype: *dict*

```


## Stopping criterion:

To stop the iteration, the following two stopping criteria must be fulfilled.

``eps1`` criterion: 
: The first stopping criterion controls the change in the reliability index as the iteration progresses. This stopping criterion is fulfilled if the weighted change from one iteration step to the next is below the threshold defined by ``eps1``.

``eps2`` criterion: 
: The second stopping criterion controls the value of the limit-state function at the current iteration step. This stopping criterion is fulfilled if the weighted value of the limit-state function is smaller than ``eps2``.

## Finite difference step size:

During the iteration, the first derivative of the limit-state function in original space has to be calculated. 
This derivative is approximated using finite differences, where the step size is obtained using the following equation:

$$
\Delta x_i = \max\left(\mathrm{fdstep}\cdot x, y\cdot\mathrm{epsdyf}\cdot\sqrt{\varepsilon},\mathrm{dxmin}_i\right)
$$

where $\varepsilon$ is the machine precision, and $x_i$ is the $i$th coordinate in original space.
        
## Example Application

For an example application of FORM, please see {ref}`the application example of Line Sampling<content:reliability_analysis:LS:example>`.


## Alternative FORM-implementation
An alternative implementation of FORM is available as function ``perform_FORM`` in the module ``fesslix.ra_form``.
This implementation uses algorithms from [SciPy][scipy] for solving the constraint optimization problem of FORM.

### Syntax

```{eval-rst}
.. function:: fesslix.ra_form.perform_FORM

    Syntax:
        ``fesslix.ra_form.perform_FORM(LSF, SAMPLER, CONFIG)``

    Description:
        Perform a run of FORM (First Order Reliability Method).
        
    :param LSF: A Python-callable that does not require any parameters and returns a float is expected.
    :type LSF: *callable*
    :param SAMPLER: A sampler for a set (or multiple sets) of random variables.
    :type SAMPLER: :class:`flx.sampler`
    :param CONFIG: 
        A configuration dictionary. The following keys are allowed in ``CONFIG``:
    
         - ``u0`` (*numpy.ndarray*): The start point of the optimization in the underlying Standard Normal space. By default, the zero-vector in standard Normal space is used as start point.
         - ``method`` (*string*, default: ``"COBYLA"``): 
         
             A parameter that specifies the optimization algorithm to use. The following values are allowed:   
             
             - ``COBYLA``
             - ``SLSQP``
             
         - ``fd_eps`` (*float*): Parameter of the SLSQP-algorithm. Value must be positive.
         - ``rhobeg`` (*float*): Parameter of the COBYLA-algorithm. Value must be positive.
         - ``print_lsf`` (*bool*, default: *True*): Output information at each call of the limit-state function.

    :type CONFIG: *dict*
    
    :returns:     
      A Python *dict* with the following information is returned:

        - ``pf`` (*float*): The probability of failure estimated by FORM.
        - ``beta`` (*float*): The reliability index estimated by FORM.
        - ``u_star`` (*numpy.ndarray*): Vector of the design-point in standard Normal space.
        
    :rtype: *dict*

```

### Application Example

In [1]:
import fesslix as flx
flx.load_engine()
## ==============================================
## Generate input model
## ==============================================
config_rv_R = { 'name':'R', 'type':'logn', 'mu':6., 'sd':1. }
config_rv_S = { 'name':'S', 'type':'normal', 'mu':1., 'sd':1. }
rv_set = flx.rv_set( {'name':'rv_set', 'allow_x2y':True}, [ config_rv_R, config_rv_S ] )
sampler = flx.sampler(['rv_set'])
## ==============================================
## Limit-state function
## ==============================================
lsf_expr = "rbrv(rv_set::R)-rbrv(rv_set::S)"
## ==============================================
## FORM solution (Python optimizer)
## ==============================================
import fesslix.ra_form
def lsf():
    return flx.eval_fun( lsf_expr )
config = { 'method':'SLSQP', 'fd_eps':1e-2 }
form_res = fesslix.ra_form.perform_FORM(lsf, sampler, config)
print(form_res)

Random Number Generator: MT19937 - initialized with rand()=553573562;
Random Number Generator: MT19937 - initialized with 1000 initial calls.
4.9183635429928625 [0. 0.] [5.91836354 1.        ]
4.9183635429928625 [0. 0.] [5.91836354 1.        ]
4.9183635429928625 [0. 0.] [5.91836354 1.        ]
4.92816810676907 [0.01 0.  ] [5.92816811 1.        ]
4.908363542992863 [0.   0.01] [5.91836354 1.01      ]
0.8155344481236915 [-3.46847277  1.5176773 ] [3.33321175 2.5176773 ]
0.8155344481236915 [-3.46847277  1.5176773 ] [3.33321175 2.5176773 ]
0.8210563608827814 [-3.45847277  1.5176773 ] [3.33873366 2.5176773 ]
0.8055344481236917 [-3.46847277  1.5276773 ] [3.33321175 2.5276773 ]
0.0003103673181259481 [-3.54622109  2.2902798 ] [3.29059017 3.2902798 ]
0.0003103673181259481 [-3.54622109  2.2902798 ] [3.29059017 3.2902798 ]
0.005761671706867144 [-3.53622109  2.2902798 ] [3.29604147 3.2902798 ]
-0.009689632681873839 [-3.54622109  2.3002798 ] [3.29059017 3.3002798 ]
0.010067003254091489 [-3.07490203  